<a href="https://colab.research.google.com/github/Firojpaudel/GenAI-Chronicles/blob/main/Seq2Seq/Seq2Seq_BART_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>